# M3GNetとASEを用いた分子シミュレーション
## 講習会前の設定 M3GNetのインストール
<BR>

参考文献（M3GNet）<BR>
- GITHUB https://github.com/materialsvirtuallab/m3gnet
- Chen, C., Ong, S.P. A universal graph deep learning interatomic potential for the periodic table. Nat Comput Sci 2, 718–728 (2022). https://doi.org/10.1038/s43588-022-00349-3. (論文引用先)
  
作成：中山将伸 (2025.3.1)  

## Case 1  M3GNet ASE ライブラリーのインストール

python3.8, 3.9が推奨<BR>
M3GNet.pyは数年前で開発が終了しているため、比較的古い環境が必要です。現在使用している他のプログラムが使えなく可能性があるので、conda, venvなどを使って、m3gnet用の環境を整えることを推奨します。（配布した動画を参照） <BR>

上記を確認したら、 直下のBOXでShift+Returnでライブラリーをインストールしてください。<BR>
注意：すでに自身の環境を持っている人は、ライブラリーのインストールによって環境が変更されてしまう可能性があります。<BR>



In [2]:
#python version確認
!python --version

zsh:1: command not found: python


In [5]:
# python3.8 または python3.9を使ってください
!pip install ase==3.23.0
!pip install m3gnet==0.2.4
!pip install matplotlib==3.7.5
!pip install nglview==3.0.3
!pip install scipy==1.10.1
!pip install tensorflow==2.12.1
# # keras 3.0以上 が入っていると動作しない可能性が高いので、消去し、2.4.3を再インストール  (tensorflowを入れるとkeras)
!pip install keras==2.4.3 --force-reinstall
!pip install scikit-learn
!pip install pymatgen==2023.8.10
!pip install numpy==1.24.4
!pip install tensorflow==2.9.1



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.0rc1 requires keras<2.14,>=2.13.1rc0, but you have keras 3.9.1 which is incompatible.
tensorflow-macos 2.13.0rc1 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.16.2 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached matplotlib-3.7.5-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.7 kB)
Using cached matplotlib-3.7.5-cp311-cp311-macosx_11_0_arm64.w

### ライブラリー動作の確認
インストールが正常に行われたかどうかを確認します。直下のBOXで Shift+Returnキーを押して、何も出力されずにインポートがされたらインストール成功です。エラーメッセージが出たら、 chatGPTなどのAIにメッセージを放り込んで、解決方法を確認すると高確率で修正法を教えてくれます。

In [ ]:
import ase 
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, UnitCellFilter
from ase.optimize import LBFGS
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md import MDLogger
from ase import Atoms
from ase.io import read, write
from ase.io import Trajectory
from ase import units
from ase.build import bulk, make_supercell
from ase.visualize import view

from m3gnet.models import M3GNet, M3GNetCalculator, Potential
potential = Potential(M3GNet.load()) #M3GNetのデフォルトポテンシャルを導入　（自分で作ったポテンシャルは要別途指定）
calculator = M3GNetCalculator(potential=potential, stress_weight=0.01)

import numpy as np
import math, random

import os,sys,csv,glob,shutil,re,time
from joblib import Parallel, delayed
args = sys.argv

# sklearn
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt


# import matgl
# from matgl.ext.ase import M3GNetCalculator
# potential = matgl.load_model("M3GNet-MP-2021.2.8-PES")
# calculator = matgl.ext.ase.M3GNetCalculator(potential=potential, stress_weight=0.01)


## Case 2 matGLを使う場合
M3GNet.pyではなく、matGL(M3GNetが内包されています)を使う場合は以下よりインストールしてください。python 3.11以降を推奨<BR>
MatGL:  https://matgl.ai/　


In [4]:
# # 以下 matglの場合
!pip install ase==3.23.0
!pip install matgl==1.1.3
!pip install matplotlib==3.10.0
!pip install numpy==1.26.4 --user
!pip install pymatgen==2024.5.1 --user
!pip install pytorch-lightning==2.0.9 --user
!pip install scipy==1.14.1
!pip install torch==2.2.1 --user
!pip install dgl==2.2.1


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached matplotlib-3.10.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached numpy-2.2.4-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.3/223.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 4.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 4.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 16.0 MB/s eta 0:00:00a 0:00:01
Using cached matplotlib-3.10.1-cp311-cp311-macosx_11_0_arm64.whl (8.0 MB)
Using cached numpy-2.2.4-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.5
    U

### ライブラリー動作の確認
インストールが正常に行われたかどうかを確認します。直下のBOXで Shift+Returnキーを押して、何も出力されずにインポートがされたらインストール成功です。エラーメッセージが出たら、 chatGPTなどのAIにメッセージを放り込んで、解決方法を確認すると高確率で修正法を教えてくれます。

In [ ]:
import ase 
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, UnitCellFilter
from ase.optimize import LBFGS
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md import MDLogger
from ase import Atoms
from ase.io import read, write
from ase.io import Trajectory
from ase import units
from ase.build import bulk, make_supercell
from ase.visualize import view

import matgl
from matgl.ext.ase import M3GNetCalculator
potential = matgl.load_model("M3GNet-MP-2021.2.8-PES")
calculator = matgl.ext.ase.M3GNetCalculator(potential=potential, stress_weight=0.01)

import numpy as np
import math, random

import os,sys,csv,glob,shutil,re,time
from joblib import Parallel, delayed
args = sys.argv

# sklearn
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

